<a href="https://colab.research.google.com/github/avtelor/avtelor/blob/main/DealFlowSlides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

used the following resources:

[python-pptx documentation Working with tables](https://python-pptx.readthedocs.io/en/latest/user/table.html)


[Turn Your Excel Sheet into a PowerPoint Presentation in Just a Couple of Minutes](https://python.plainenglish.io/turn-your-excel-sheets-into-a-powerpoint-presentation-just-in-a-couple-of-minutes-62ae4b8bc13b)

[Code to resize pic to fit the place holder](https://pythonprogramming.altervista.org/inserting-an-image-in-powerpoint-with-python/?doing_wp_cron=1661671938.5373210906982421875000)

[Creating executable from Python (1)](https://www.tomshardware.com/how-to/create-python-executable-applications)

[Creating executable from Python (2)](https://pythonguides.com/convert-python-file-to-exe-using-pyinstaller/)

In [1]:

!pip install python-pptx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 5.0 MB/s 
     |████████████████████████████████| 149 kB 41.0 MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470951 sha256=73fa45ed295a4e37d513a607eef158bcce881cd466d9d72ea5e222a6f92a1a13
  Stored in directory: /root/.cache/pip/wheels/a7/ab/f4/52560d0d4bd4055e9261c6df6e51c7b56c2b23cca3dee811a3
Successfully built python-pptx


In [2]:
#import pptx
from pptx import Presentation
prs  = Presentation('Dealflow_template.pptx')
slide = prs.slides.add_slide(prs.slide_layouts[1])
#-----------------------
for s_ in slide.placeholders:
    print('ID = {0:d}, Shape name is \
          "{1:s}"'.format(s_.placeholder_format.idx,s_.name))

ID = 1, Shape name is           "Content Placeholder 1"
ID = 10, Shape name is           "Text Placeholder 2"
ID = 11, Shape name is           "Picture Placeholder 3"
ID = 13, Shape name is           "Picture Placeholder 4"
ID = 15, Shape name is           "Table Placeholder 5"
ID = 16, Shape name is           "Content Placeholder 6"
ID = 17, Shape name is           "Content Placeholder 7"
ID = 18, Shape name is           "Content Placeholder 8"
ID = 19, Shape name is           "Picture Placeholder 9"
ID = 20, Shape name is           "Table Placeholder 10"
ID = 21, Shape name is           "Content Placeholder 11"


Loading the XLS workbook and loading the cells int

In [8]:
from pptx.util import Inches
from pptx.util import Inches, Pt
from pptx.dml.color import ColorFormat, RGBColor
#from pptx.enum.dml import MSO_COLOR_TYPE, MSO_THEME_COLOR
import openpyxl
from openpyxl import load_workbook
import os.path
from PIL import Image

filename="Dealflow.xlsx"
#print (filename)
wb = openpyxl.load_workbook(filename, data_only = True)
#print ('Sheet:',wb.sheetnames )
#ws = wb.get_sheet_by_name(name = 'big_data')

def iter_cells(table):
    for row in table.rows:
        for cell in row.cells:
            yield cell



def resize_image(image_url, placeholder):
    
    # Extract the image size
    im = Image.open(image_url)
    width, height = im.size
    
    # Make sure the placeholder doesn't zoom in
    placeholder.height = height
    placeholder.width = width
    
    # Insert the picture
    placeholder = placeholder.insert_picture(image_url)

    # Calculate ratios and compare
    image_ratio = width / height
    placeholder_ratio = placeholder.width / placeholder.height
    ratio_difference = placeholder_ratio - image_ratio
 
    # Placeholder width too wide:
    if ratio_difference > 0:
        difference_on_each_side = ratio_difference / 2
        placeholder.crop_left = -difference_on_each_side
        placeholder.crop_right = -difference_on_each_side
    # Placeholder height too high
    else:
        difference_on_each_side = -ratio_difference / 2
        placeholder.crop_bottom = -difference_on_each_side
        placeholder.crop_top = -difference_on_each_side


ws = wb.worksheets[0]
ppt = Presentation('Dealflow_template.pptx')
layout=1 # layout 'ID=0'- basic layout ; "1" - extended
layout_company_overview = ppt.slide_layouts[layout]  
for slidenum in range (1,20) : 
  new_slide = ["new_slide" + str(slidenum)] 
  new_slide = ppt.slides.add_slide(layout_company_overview)
  new_slide.placeholders[10].text  = str(ws['C'+str(slidenum+1)].value) # Name
  new_slide.placeholders[21].text  = str(ws['U'+str(slidenum+1)].value).replace(", ", "\n") # Founders  
  for paragraph in new_slide.shapes.placeholders[21].text_frame.paragraphs:
        paragraph.font.size = Pt(13)
  new_slide.placeholders[1].text  = str(ws['H'+str(slidenum+1)].value)  # Essence
  new_slide.placeholders[16].text  = str(ws['I'+str(slidenum+1)].value) # Differentiation
  new_slide.placeholders[17].text  = str(ws['J'+str(slidenum+1)].value) # Intel Touchpoints
  new_slide.placeholders[18].text  = str(ws['K'+str(slidenum+1)].value) # Value to Intel

  pic_file_name='/content/Pics/'+str(ws['C'+str(slidenum+1)].value)+'1.png'
  file_exists = os.path.exists(pic_file_name)
  if file_exists: 
    #new_slide.placeholders[11].insert_picture(pic_file_name)    
    resize_image(pic_file_name, new_slide.placeholders[11])
    pic_file_name='/content/Pics/'+str(ws['C'+str(slidenum+1)].value)+'2.png'
    file_exists = os.path.exists(pic_file_name)
    if file_exists: 
      #new_slide.placeholders[13].insert_picture(pic_file_name)
      resize_image(pic_file_name, new_slide.placeholders[13])
  
  shape = new_slide.placeholders[15].insert_table(rows=2, cols=5)
  table=shape.table
  table.cell(0,0).text="Status"
  table.cell(0,1).text="Pillar"
  table.cell(0,2).text="Synergy"
  table.cell(0,3).text="Funding"
  table.cell(0,4).text="Score"
  table.cell(1,0).text=str(ws['E'+str(slidenum+1)].value) # status
  table.cell(1,1).text=str(ws['F'+str(slidenum+1)].value) # Pillars
  table.cell(1,2).text=str(ws['G'+str(slidenum+1)].value) # Synergy
  table.cell(1,3).text=str(ws['M'+str(slidenum+1)].value) # Funding  
  table.cell(1,4).text=str(ws['X'+str(slidenum+1)].value) # Score
  
  for cellnum in range (0,5):
  #  fill=table.cell(0,cellnum).fill
  #  fill = cell.fill
  #  fill.solid()
  #  fill.fore_color.rgb = RGBColor(0xE6, 0xF2, 0xFF)
    cell=table.cell(0,cellnum)
    for paragraph in cell.text_frame.paragraphs:
        paragraph.font.size = Pt(16) if layout==0 else Pt(12)
        #paragraph.font.bold = True        
        paragraph.font.color.rgb = RGBColor(0xFF, 0xff, 0xff)

  for cellnum in range (0,5):
  #  fill=table.cell(1,cellnum).fill
  #  fill = cell.fill
  #  fill.solid()
  #  fill.fore_color.rgb = RGBColor(0xE6, 0xF2, 0xFF)
    cell=table.cell(1,cellnum)
    for paragraph in cell.text_frame.paragraphs:
        paragraph.font.size = Pt(14) if layout==0 else Pt(10)
        paragraph.font.color.rgb = RGBColor(0x0, 0x33, 0x66)

  if layout==1 :
    shape = new_slide.placeholders[20].insert_table(rows=2, cols=6)
    table=shape.table
    table.cell(0,0).text="Team"
    table.cell(0,1).text="Coachability"
    table.cell(0,2).text="Impact"
    table.cell(0,3).text="Success"
    table.cell(0,4).text="technology"
    table.cell(0,5).text="Maturity"    
    table.cell(1,0).text=str(ws['O'+str(slidenum+1)].value) # Team
    table.cell(1,1).text=str(ws['P'+str(slidenum+1)].value) # Coachability
    table.cell(1,2).text=str(ws['Q'+str(slidenum+1)].value) # Impact
    table.cell(1,3).text=str(ws['R'+str(slidenum+1)].value) # Success
    table.cell(1,4).text=str(ws['S'+str(slidenum+1)].value) # Technology
    table.cell(1,5).text=str(ws['T'+str(slidenum+1)].value) # Maturity
  
    for cellnum in range (0,6):
    #  fill=table.cell(0,cellnum).fill
    #  fill = cell.fill
    #  fill.solid()
    #  fill.fore_color.rgb = RGBColor(0xE6, 0xF2, 0xFF)
      cell=table.cell(0,cellnum)
      for paragraph in cell.text_frame.paragraphs:
        paragraph.font.size = Pt(12)
        #paragraph.font.bold = True        
        paragraph.font.color.rgb = RGBColor(0xFF, 0xff, 0xff)

    for cellnum in range (0,6):
    #  fill=table.cell(1,cellnum).fill
    #  fill = cell.fill
    #  fill.solid()
    #  fill.fore_color.rgb = RGBColor(0xE6, 0xF2, 0xFF)
      cell=table.cell(1,cellnum)
      for paragraph in cell.text_frame.paragraphs:
        paragraph.font.size = Pt(10)
        paragraph.font.color.rgb = RGBColor(0x0, 0x33, 0x66)
  
ppt.save("Dealflow.pptx")
print ('slides are ready')

slides are ready
